<h1>Cleaning the Names of the Data</h1>

Import the library to read the excel

In [2]:
import pandas as pd
import numpy as np

Read the excel into the code

In [4]:
df=pd.read_excel('RawData/Ranking.xlsx', sheet_name='Consolidated2')
df = df[df['NAME'].notna()]
display(df)

,Unnamed: 0,Date,Day,Month,Year,Event,NAME,Age,TEAM,Result,Unnamed: 10,m/s,Competition,Year D.O.B.,"Info, if any",Date of Birth (PDPA),Unnamed: 16
0,NaN,2023-01-14 00:00:00,14,1.0,2023.0,Men High Jump,"KAM, KAMPTON",22,INDIVIDUAL,2.06,NaN,-,International - Penn 10-Team Select,2000,"Sanctioned,1pos",-,NaN
1,NaN,2023-01-04 00:00:00,4,1.0,2023.0,Men Long Jump,"TOH, WEI YU",-,NANYANG TECHNOLOGICAL UNI,6.64,NaN,0.2,Local - IVP,-,-,-,NaN
2,NaN,2023-01-04 00:00:00,4,1.0,2023.0,Men Long Jump,"MUHAMAD NURSHAHILMI, BIN NORADEE",-,TEMASEK POLYTECHNIC,6.28,NaN,0.1,Local - IVP,-,-,-,NaN
3,NaN,2023-01-04 00:00:00,4,1.0,2023.0,Men Long Jump,"SEBASTIAN, KIRAN",-,REPUBLIC POLYTECHNIC,6.27,NaN,1.7,Local - IVP,-,-,-,NaN
4,NaN,2023-01-04 00:00:00,4,1.0,2023.0,Men Long Jump,"NAING, KYAW LINN",-,SINGAPORE MANAGEMENT UNI,6.26,NaN,1.2,Local - IVP,-,-,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122793,NaN,2006-07-17 00:00:00,17,7.0,2006.0,Men 100 Meter Dash,Azhari Muhd Yusof,17,Singapore,11.12,NaN,-0.4,International - Asian Junior,1989,"Semis, 7/8pos",-,NaN
122794,NaN,2006-07-17 00:00:00,17,7.0,2006.0,Women Discus Throw (1kg),"Wan, Lay Chi",18,Singapore,44.85,NaN,-,International - Asian Junior,1988,"Final, 6/14pos",-,NaN
122795,NaN,2006-07-17 00:00:00,17,7.0,2006.0,Women 100 Meter Dash,Ann Siao Mei,19,Singapore,12.93,NaN,-3,International - Asian Junior,1987,"R1, 5/7pos",-,NaN
122796,NaN,2006-07-17 00:00:00,17,7.0,2006.0,Women 100 Meter Dash,"Choo Sze-Min, Amanda",19,Singapore,12.39,NaN,0.2,International - Asian Junior,1987,"R1, 4/6pos q",-,NaN


Split all names into first and last name, remove the relays

In [5]:
dffnln=df.copy()
dffnln=dffnln.loc[~(dffnln['Event'].isin(['Women 4x400 Meter Relay', ' Men 4x400 Meter Relay', 'Men 4x400 Meter Relay', 'Mixed 4x400 Meter Relay', 'Women 4x100 Meter Relay', 'Men 4x100 Meter Relay', 'Mixed 4x100 Meter Relay']))]
dffnln.Event.unique()

dffnln['FirstName'] = dffnln['NAME'].str.split(',').str[0]
dffnln['LastName'] = dffnln['NAME'].str.split(',').str[-1]



# writer = pd.ExcelWriter('Checkpoints.xlsx', engine = 'openpyxl')
# dffnln.to_excel(writer, sheet_name = 'FirstNameLastName')
# writer.close()

# dffnln.to_csv('Checkpoint1.csv')

Machine Learning to check the similarities between names

In [6]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
    # print vec1, vec2
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    return Counter(WORD.findall(text))

def get_similarity(a, b):
    a = text_to_vector(a.strip().lower())
    b = text_to_vector(b.strip().lower())

    return get_cosine(a, b)

get_similarity('TAN ELIZABETH-A', 'TAN ELIZABETH ANN SHEE RU') # returns 0.9258200997725514

0.5163977794943222

See all similar names

In [ ]:
any(map(lambda v: v in ('BIN ZAINAL MUHAMMAD FAZREY'.lower()).split(' '), ['mohamad', 'muhammad', 'mohammad', 'muhammad']))

True

In [ ]:
# ToCheck={}
# dfname=dffnln.copy()
# # dfname=dfname.drop_duplicates(subset=['NAME'])
# # dfname.to_csv('dfname.csv')

# counter=0

# for firstname, lastname in zip(dfname['FirstName'], dfname['LastName']):
#     counter+=1
#     for keys in ToCheck.keys():
#         i, j=keys.split('/')[0], keys.split('/')[-1]
#         score1=get_similarity(firstname,i)
#         score2=get_similarity(lastname, j)
#         if any(map(lambda v: v in (firstname.lower()+lastname.lower()).split(' '), ['mohamad', 'muhammad', 'mohammad', 'muhammad'])): 
#             # print(firstname, lastname)
#             if score1>0.7 and score2>0.7:
#                 ToCheck[keys]+=[f'{firstname}/{lastname} ({round(score1, 2), round(score2, 2)})']
#                 # print(f'{firstname} // {i}')
#                 break
#             else:
#                 scoreboth=get_similarity(firstname+lastname,keys)
#                 if scoreboth>0.7:
#                     ToCheck[keys]+=[f'{firstname}/{lastname} ({round(scoreboth, 2)})']
#                     break
#         else:
#             if score1>0.5 and score2>0.5:
#                 ToCheck[keys]+=[f'{firstname}/{lastname} ({round(score1, 2), round(score2, 2)})']
#                 # print(f'{firstname} // {i}')
#                 break
#             else:
#                 scoreboth=get_similarity(firstname+lastname,keys)
#                 if scoreboth>0.5:
#                     ToCheck[keys]+=[f'{firstname}/{lastname} ({round(scoreboth, 2)})']
#                     break
#     else:
#         ToCheck[(firstname+'/'+lastname).upper()]=[]

#     if counter%500==0:
#         print(f'{counter} out of {len(dfname)} completed.')

# pd.DataFrame(pd.Series(ToCheck)).to_csv('Checkpoint6.csv')


In [ ]:
# UniqueNames=[''] ##Can consider dropping the mohammad when comparing or dropping punctuation
# counter=0
# checkdf=dffnln.copy()
# checkdf=checkdf.drop_duplicates(subset=['NAME'])
# checkdf.to_csv('checkpoint3.csv')


# for name, team, age in zip(checkdf['NAME'], checkdf['TEAM'], checkdf['Age']):
#     # print(i)
#     counter+=1
#     for j in UniqueNames:
#         score=get_similarity(name,j)
#         if 'MOHAMMAD' in name:
#             if score > 0.65: #Check if the names is already in the UniqueNames, if similarity score >60% means same name
#                 if len(name)>=len(j): #Take the longer name for the Unique Names
#                     # with open("output.txt", "a") as f:
#                     #     print(f"Replacing {j} ({len(j)}) with {i} ({len(i)}). {counter}", file=f)
#                     # UniqueNameReplace=bool(input(f"Replace {name.upper()} with {j.upper()}.?"))
#                     # if UniqueNameReplace:
#                     UniqueNames[UniqueNames.index(j)]=name
#                 break
#         else:
#             if score > 0.70: #Check if the names is already in the UniqueNames, if similarity score >60% means same name
#                 if len(name)>=len(j): #Take the longer name for the Unique Names
#                     # with open("output.txt", "a") as f:
#                     #     print(f"Replacing {j} ({len(j)}) with {i} ({len(i)}). {counter}", file=f)
#                     # UniqueNameReplace=bool(input(f"Replace {name.upper()} with {j.upper()}.?"))
#                     # if UniqueNameReplace:
#                     UniqueNames[UniqueNames.index(j)]=name
#                 break
            
#     else:
#         # with open("output.txt", "a") as f:
#         #     print(f"Adding {i} into Unique Names.", file=f)

#         UniqueNames.append(name)
    
#     if counter%500==0:
#         print(f'{counter} out of {len(checkdf)} done.')

# print(len(UniqueNames))
# print(ToCheck)

# Checking=pd.DataFrame(UniqueNames)
# Checking.to_csv('Checkpoint5.csv')
    



In [ ]:
# UniqueNames={'Name': 'Team'}
# counter=0
# checkdf=dffnln.copy()
# checkdf=checkdf.drop_duplicates(subset=['NAME'])
# checkdf.to_csv('checkpoint3.csv')
# for name, team, age in zip(checkdf['NAME'], checkdf['TEAM'], checkdf['Age']):
#     # print(i)
#     counter+=1
#     for j in UniqueNames:
#         score=get_similarity(name,j)
#         if 'MOHAMMAD' in name:
#             if score > 0.65 and team==UniqueNames[j]: #Check if the names is already in the UniqueNames, if similarity score >60% means same name
#                 if len(name)>=len(j): #Take the longer name for the Unique Names
#                     # with open("output.txt", "a") as f:
#                     #     print(f"Replacing {j} ({len(j)}) with {i} ({len(i)}). {counter}", file=f)
#                     UniqueNameReplace=bool(input(f"Replace {name.upper()} with {j.upper()}.?"))
#                     if UniqueNameReplace:
#                         UniqueNames[j]=name.upper()

#                 break
#         else:
#             if score > 0.70: #Check if the names is already in the UniqueNames, if similarity score >60% means same name
#                 if len(name)>=len(j) and team==UniqueNames[j]: #Take the longer name for the Unique Names
#                     # with open("output.txt", "a") as f:
#                     #     print(f"Replacing {j} ({len(j)}) with {i} ({len(i)}). {counter}", file=f)
#                     UniqueNameReplace=bool(input(f"Replace {name.upper()} with {j.upper()}.?"))
#                     if UniqueNameReplace:
#                         UniqueNames[j]=name.upper()

#                 break
            
#     else:
#         # with open("output.txt", "a") as f:
#         #     print(f"Adding {i} into Unique Names.", file=f)

#         UniqueNames[name]=team
    
#     if counter%500==0:
#         print(f'{counter} out of {len(checkdf)} done.')

# print(len(UniqueNames))

# Checking=pd.DataFrame({'Name':[i for i in UniqueNames.keys()], 'Team': [j for j in UniqueNames.values()]})
# Checking.to_csv('Checkpoint5.csv')
    
    



In [ ]:
# dfUniqueName=dffnln.copy()
# EditedNames=[]
# counter=0

# for i in dfUniqueName['NAME']:
#     counter+=1
#     # print(i)
#     for j in UniqueNames:
#         score=get_similarity(i,j)
#         if 'MOHAMMAD' in i:
#             if score > 0.65: #Check if the names is already in the UniqueNames, if similarity score >60% means same name
#                 EditedNames.append(j)
#                 break
#         else:
#             if score > 0.55:
#                 EditedNames.append(j)
#                 break
#     else:
#         EditedNames.append('No Match')
    
#     # print(f'i is {i}, j is {j}')

#     if counter%500==0:
#         print(f'{counter} out of {len(dfUniqueName)} done.')

# dfUniqueName['Edited Names']=EditedNames

# dfUniqueName.to_csv('Checkpoint4.csv')

Assign UniqueID to all names

In [ ]:
# # UniqueNames={}
# # counter=0
# # for key,value in dffnln['NAME'].items():
# #     if value not in UniqueNames:
# #         UniqueNames[value]=counter
# #         counter+=1

# dffnln['UniqueID']=(dffnln['FirstName'].str[:2]+dffnln['LastName'].str[:2]+dffnln['Year D.O.B.'].astype(str)).str.replace(' ', '')    
# # writer = pd.ExcelWriter('Checkpoints.xlsx', engine = 'openpyxl')
# # dffnln.to_excel(writer, sheet_name = 'UniqueID')
# # writer.close()

# dffnln.to_csv('Checkpoint2.csv')





In [ ]:
# UniqueNames=pd.read_csv('UniqueNames.csv')
# Changer=[]

# UniqueNames.columns=['Name', 'Edit']
# # check=UniqueNames['Edit'].tolist()
# # Changer=[]
# for names in UniqueNames['Edit']:
#     # print(names)
#     newname=re.sub( "\s*\(.*?\)\s*"," ",names)
#     newname=newname.replace('[]','')
#     newname=newname.replace('[','')
#     newname=newname.replace(']', '')
#     newname=newname.replace('(', '')
#     newname=newname.replace(')', '')
#     newname=newname.replace("'", '')
#     newname=newname.replace("'", '')
#     newname=newname.replace('/', '')
#     Changer.append(newname)
#     # print(newname)
#     # Changer.append(re.sub( "\s*\(.*?\)\s*"," ",names))
#     # Changer.append(name.replace('[]', ''))

# UniqueNames['NameFix']=UniqueNames['Name'].replace(['/', "'", '(', ')'],'')
# UniqueNames['ChangetoThese']=Changer
# UniqueNames.to_csv('UniqueNamesEdited.csv')


In [ ]:
# Namedb=pd.read_csv('UniqueNamesRev13.csv')
# Namedb=Namedb.fillna('EMPTY')
# # newdf=pd.read_csv('dfname.csv')
# newdf=dffnln.copy()
# toreplace=[]
# formetocheck={}
# counter=0

# # for k in Namedb.keys():
# #     print(Namedb[k])

# # print(True in Namedb['TAN ELIZABETH ANN SHEE RU'].str.contains('TAN ELIZABETH ANN SHEE RU' ))
# # for i in Namedb.keys():
# #     print('KAM KAMPTON' == i)
# # print('SO GOTHANDARAMAN BHUVANESH' in Namedb.items())

# # for k, v in Namedb.items():
# #     if 'SO GOTHANDARAMAN/ BHUVANESH' in v: 
# #         print('yes')
# #         break

# # print('KAM KAMPTON' in Namedb.keys())

# newdf['CheckNames']=newdf['Name'].replace(',', '', regex=True)
# newdf['CheckNames']=newdf['Name'].replace("'", '', regex=True)
# newdf['CheckNames']=newdf['CheckNames'].str.upper()
# newdf['CheckNames']=newdf['CheckNames'].str.strip()
# # print(newdf['CheckNames'])
# for names in newdf['CheckNames']:
#     names=names.replace(',', '')
#     names=names.strip()
#     for k in Namedb.keys():
#         try:
#             if names.upper()==k:
#                 toreplace.append(k)
#                 formetocheck[names]=k
#                 break
#             elif (Namedb[k].str.match(names.upper()).any()): #please remove all the unnamed rows if get the .str accessor error
#                 toreplace.append(k)
#                 formetocheck[names]=k
#                 break
#         except:
#             toreplace.append('Error')
#             formetocheck[names]='Error'
#             break
#     else:
#         toreplace.append('No Match')
#         formetocheck[names]='No Match'
#         # print(names)

#     counter+=1
#     if counter%500==0:
#         print(f'{counter} out of {len(newdf["Name"])} done.')

# newdf['NAME EDITED']=toreplace
# newdf.to_csv('2014.csv')
 
# #Just run this and wait 30 mins to see if it works

# # Namedb.value()

In [ ]:
# pd.DataFrame(toreplace).to_csv('ToReplace.csv')
# pd.DataFrame(formetocheck).to_csv('ForMeToCheck.csv')

# newdf.to_csv('2022.csv')

In [7]:
UniqueNames=pd.read_excel('NameVariationDict/UniqueNamesRev24.xlsx')
UniqueNames=UniqueNames.fillna('NIL')
UniqueNames.reset_index()
newdf=dffnln.copy() #Copy the dataframe to prevent any alteration to original file
counter=0 #To see the progress of the code
toreplace=[]

#Create a column called CheckNames to read all the names without the commas. Make all captial also
newdf['CheckNames']=newdf['NAME'].replace(',', '', regex=True)
newdf['CheckNames']=newdf['CheckNames'].str.upper()
newdf['CheckNames']=newdf['CheckNames'].str.strip()
print(len(newdf))



for names in newdf['CheckNames']:
    if counter%500==0:
        print(f'{counter} out of {len(newdf["CheckNames"])} done.')
    
    
    try:
        if UniqueNames['Actual Names'].str.contains(names).any():
            row_num=[i for i, x in enumerate(UniqueNames['Actual Names'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var1'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var1'] == names].index
            row_num=[i for i, x in enumerate(UniqueNames['Var1'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var2'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var2'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var2'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var3'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var3'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var3'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var4'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var4'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var4'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var5'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var5'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var5'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var6'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var6'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var6'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var7'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var7'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var7'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var8'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var8'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var8'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var9'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var9'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var9'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var10'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var10'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var10'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var11'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var11'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var11'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var12'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var12'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var12'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var13'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var13'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var13'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var14'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var14'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var14'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var15'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var15'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var16'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var16'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var17'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var17'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var18'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var18'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var19'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var19'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var20'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var20'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var21'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var21'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var22'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var22'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var23'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var23'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var24'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var24'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var25'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var25'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var26'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var26'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var27'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var27'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var28'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var28'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var29'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var29'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var30'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var30'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var31'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var31'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var32'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var32'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var33'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var33'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var34'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var34'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var35'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var35'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var36'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var36'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        elif UniqueNames['Var37'].str.contains(names).any():
            # row_num = UniqueNames[UniqueNames['Var15'] == names].index 
            row_num=[i for i, x in enumerate(UniqueNames['Var37'].str.contains(names)) if x]
            toreplace.append(UniqueNames['Actual Names'][row_num[0]])
        else:
            toreplace.append('NoMatch')
        counter+=1
    except:
        toreplace.append('Error')
        counter+=1

#Replace all the original names to the correct names
newdf['NAME EDITED']=toreplace

#Remember to rename the file everytime it is produced
newdf.to_excel('CONSOLIDATED.xlsx')

print("done")

    


        

FileNotFoundError: [Errno 2] No such file or directory: 'UniqueNamesRev24.xlsx'

In [ ]:
UniqueNames['Actual Names'].str.contains('TAN ELIZABETH-ANN').any()

True